## Testing scrapling MCP usage

In [7]:
import sys
sys.path.append("..")
from langchain.agents import create_agent
from models import model

import asyncio
from langchain_mcp_adapters.client import MultiServerMCPClient  

client = MultiServerMCPClient(  
        {
            "scraper": {
                "transport": "http",  # HTTP-based remote server
                # Ensure you start your weather server on port 8000
                "url": "http://localhost:8000/mcp",
            }
        }
    )

In [8]:
tools = await client.get_tools() 

sys_prompt = \
    """
    You are Scraper, and AI Agent specialized in web scraping and research. You have access to a group
    of tools for web scraping.
    Answer the user answer with kind and formal tone.
    """

agent = create_agent(model=model, tools=tools, system_prompt=sys_prompt)

In [52]:
import json

from langgraph.checkpoint.memory import MemorySaver

checkpointer = MemorySaver()
agent.checkpointer = checkpointer

config = {
    "configurable": {
        "thread_id": "0",
    },
    "recursion_limit": 100,

}

input_message = "- In the video https://www.youtube.com/watch?v=L1vXCYZAYYM, what is the highest number of bird species to be on camera simultaneously?"
async for event in agent.astream({"messages": [input_message]}, config, stream_mode="updates"):
    print(event)
    if 'message' in event.keys():
        for m in event['messages']:
            print(event.type)
            # print("#"*200 + '\n\n')
            # if m.type == 'human':
            #     print(f"User:\n{m.content}")
            # elif m.type == 'ai':
            #     for c in m.content:
            #         if c['type'] == 'text':
            #             print(f"AI:\n{c['text']}")
            #         elif c['type'] == 'tool_use':
            #             print(f"Calling tool: '{c['name']}'")
            # else:
            #     for c in m.content:
            #         if c['type'] == 'text':
            #             print(f"\n\n{c['text'][:200]}\n[...]\n\n")
            # print("\n\n")

{'model': {'messages': [AIMessage(content=[{'text': "\n\nI'll help you find information about that YouTube video. Let me fetch the video page to learn more about its content.", 'type': 'text'}, {'id': 'toolu_01SrKP7B6oQPWGgXyHzwv28h', 'input': {'url': 'https://www.youtube.com/watch?v=L1vXCYZAYYM', 'network_idle': True}, 'name': 'fetch', 'type': 'tool_use', 'caller': {'type': 'direct'}}], additional_kwargs={}, response_metadata={'id': 'msg_01TjB6SfcBhLzKguAk25Lj5p', 'model': 'claude-opus-4-6', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'inference_geo': 'not_available', 'input_tokens': 11162, 'output_tokens': 118, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-opus-4-6', 'model_provider': 'anthropic'}, id='lc_run--019ca4f5-02df-7e41-b750-19d94a83f2a7-0', tool_calls=[{'name': 'fetch', 'args': {'url

In [44]:
messages = agent.get_state(config=config).values["messages"]
for m in messages:
    print("#"*200 + '\n\n')
    if m.type == 'human':
        print(f"User:\n{m.content}")
    elif m.type == 'ai':
        for c in m.content:
            if c['type'] == 'text':
                print(f"AI:\n{c['text']}")
            elif c['type'] == 'tool_use':
                print(f"Calling tool: '{c['name']}'")
    else:
        for c in m.content:
            if c['type'] == 'text':
                print(f"\n\n{c['text'][:200]}\n[...]\n\n")
    print("\n\n")

########################################################################################################################################################################################################


User:
- In the video https://www.youtube.com/watch?v=L1vXCYZAYYM, what is the highest number of bird species to be on camera simultaneously?



########################################################################################################################################################################################################


AI:


I'll help you analyze that YouTube video! Let me first fetch the video page to get information about it.
Calling tool: 'fetch'



########################################################################################################################################################################################################




{
  "status": 200,
  "content": [
    "Back\n[![]()](/ \"YouTube Home\")\n\nBR\n\nSkip navigation\n\nSearch\n

In [56]:
tools

[StructuredTool(name='get', description='Make GET HTTP request to a URL and return a structured output of the result.\nNote: This is only suitable for low-mid protection levels. For high-protection levels or websites that require JS loading, use the other tools directly.\nNote: If the `css_selector` resolves to more than one element, all the elements will be returned.\n\n:param url: The URL to request.\n:param impersonate: Browser version to impersonate its fingerprint. It\'s using the latest chrome version by default.\n:param extraction_type: The type of content to extract from the page. Defaults to "markdown". Options are:\n    - Markdown will convert the page content to Markdown format.\n    - HTML will return the raw HTML content of the page.\n    - Text will return the text content of the page.\n:param css_selector: CSS selector to extract the content from the page. If main_content_only is True, then it will be executed on the main content of the page. Defaults to None.\n:param ma